# Fully Convolutional Interior/Edge Segmentation for 2D Data

In [2]:
# This notebook trains a featurenet model on the original data from Keren et al 2018
import os
import errno

import numpy as np

import deepcell

### Load the training data

In [10]:
# create folder for this set of experiments
experiment_folder = "featurenet_samir"
MODEL_DIR = os.path.join("/data/analyses", experiment_folder)
NPZ_DIR = "/data/npz_data/"
LOG_DIR = '/data/logs'

if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)

### Set up filepath constants

### Set up training parameters

In [13]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

conv_model_name = 'featurenet_samir'
npz_name = "samir_labels_256x256.npz"
DATA_FILE = os.path.join(NPZ_DIR, npz_name)

n_epoch = 100  # Number of training epochs
norm_method = 'whole_image'  # data normalization
receptive_field = 61  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# FC training settings
n_skips = 3  # number of skip-connections (only for FC training)
batch_size = 1  # FC training uses 1 image per batch

# Transformation settings
transform = 'pixelwise'
dilation_radius = 0  # change dilation radius for edge dilation
separate_edge_classes = False  # break edges into cell-background edge, cell-cell edge
n_features = 4 if separate_edge_classes else 3

### Next, Create a model for the edge/interior segmentation

#### Instantiate the segmentation transform model

In [14]:
from deepcell import model_zoo

conv_model = model_zoo.bn_feature_net_skip_2D(
    receptive_field=receptive_field,
    n_skips=n_skips,
    n_features=n_features,
    norm_method=norm_method,
    n_conv_filters=32,
    n_dense_filters=128,
    last_only=False,
    input_shape=(256, 256, 1))

#### Train the segmentation transform model

In [15]:
from deepcell.training import train_model_conv

conv_model = train_model_conv(
    model=conv_model,
    dataset=DATA_FILE,  # full path to npz file
    model_name=conv_model_name,
    test_size=0.1,
    seed=1,
    transform=transform,
    dilation_radius=dilation_radius,
    separate_edge_classes=separate_edge_classes,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epoch=n_epoch,
    log_dir=LOG_DIR,
    model_dir=MODEL_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    zoom_range=(0.7, 1/0.7))

X_train shape: (115, 256, 256, 1)
y_train shape: (115, 256, 256, 1)
X_test shape: (13, 256, 256, 1)
y_test shape: (13, 256, 256, 1)
Output Shape: (None, 256, 256, 3)
Number of Classes: 3
Training on 1 GPUs
Epoch 1/100
114/115 [============================>.] - ETA: 0s - loss: 3.0046 - model_5_loss: 0.7502 - model_6_loss: 0.7456 - model_7_loss: 0.7399 - model_8_loss: 0.7328 - model_5_acc: 0.7162 - model_6_acc: 0.7214 - model_7_acc: 0.7316 - model_8_acc: 0.7284
Epoch 00001: val_loss improved from inf to 2.85959, saving model to /data/analyses/featurenet_samir/featurenet_samir.h5
115/115 [==============================] - 31s 268ms/step - loss: 2.9985 - model_5_loss: 0.7487 - model_6_loss: 0.7440 - model_7_loss: 0.7385 - model_8_loss: 0.7312 - model_5_acc: 0.7169 - model_6_acc: 0.7221 - model_7_acc: 0.7322 - model_8_acc: 0.7290 - val_loss: 2.8596 - val_model_5_loss: 0.6779 - val_model_6_loss: 0.6660 - val_model_7_loss: 0.8330 - val_model_8_loss: 0.6465 - val_model_5_acc: 0.8133 - val_mode

Epoch 11/100
114/115 [============================>.] - ETA: 0s - loss: 2.2307 - model_5_loss: 0.5522 - model_6_loss: 0.5474 - model_7_loss: 0.5478 - model_8_loss: 0.5472 - model_5_acc: 0.8017 - model_6_acc: 0.8007 - model_7_acc: 0.8011 - model_8_acc: 0.8019
Epoch 00011: val_loss did not improve from 2.16959
115/115 [==============================] - 15s 134ms/step - loss: 2.2280 - model_5_loss: 0.5515 - model_6_loss: 0.5467 - model_7_loss: 0.5472 - model_8_loss: 0.5465 - model_5_acc: 0.8021 - model_6_acc: 0.8010 - model_7_acc: 0.8015 - model_8_acc: 0.8023 - val_loss: 2.1897 - val_model_5_loss: 0.5488 - val_model_6_loss: 0.5438 - val_model_7_loss: 0.5403 - val_model_8_loss: 0.5207 - val_model_5_acc: 0.8109 - val_model_6_acc: 0.8089 - val_model_7_acc: 0.8086 - val_model_8_acc: 0.8072
Epoch 12/100
114/115 [============================>.] - ETA: 0s - loss: 2.1661 - model_5_loss: 0.5365 - model_6_loss: 0.5320 - model_7_loss: 0.5311 - model_8_loss: 0.5304 - model_5_acc: 0.8068 - model_6_acc

Epoch 21/100
114/115 [============================>.] - ETA: 0s - loss: 2.1069 - model_5_loss: 0.5233 - model_6_loss: 0.5163 - model_7_loss: 0.5158 - model_8_loss: 0.5154 - model_5_acc: 0.8109 - model_6_acc: 0.8115 - model_7_acc: 0.8107 - model_8_acc: 0.8115
Epoch 00021: val_loss improved from 2.11561 to 2.09358, saving model to /data/analyses/featurenet_samir/featurenet_samir.h5
115/115 [==============================] - 16s 136ms/step - loss: 2.1075 - model_5_loss: 0.5235 - model_6_loss: 0.5165 - model_7_loss: 0.5159 - model_8_loss: 0.5155 - model_5_acc: 0.8108 - model_6_acc: 0.8114 - model_7_acc: 0.8106 - model_8_acc: 0.8114 - val_loss: 2.0936 - val_model_5_loss: 0.5166 - val_model_6_loss: 0.5147 - val_model_7_loss: 0.5172 - val_model_8_loss: 0.5091 - val_model_5_acc: 0.8129 - val_model_6_acc: 0.8030 - val_model_7_acc: 0.8058 - val_model_8_acc: 0.8068
Epoch 22/100
114/115 [============================>.] - ETA: 0s - loss: 2.0978 - model_5_loss: 0.5207 - model_6_loss: 0.5140 - model_

114/115 [============================>.] - ETA: 0s - loss: 2.0830 - model_5_loss: 0.5177 - model_6_loss: 0.5105 - model_7_loss: 0.5091 - model_8_loss: 0.5098 - model_5_acc: 0.8103 - model_6_acc: 0.8123 - model_7_acc: 0.8111 - model_8_acc: 0.8106
Epoch 00041: val_loss did not improve from 2.06013
115/115 [==============================] - 15s 133ms/step - loss: 2.0802 - model_5_loss: 0.5170 - model_6_loss: 0.5097 - model_7_loss: 0.5083 - model_8_loss: 0.5091 - model_5_acc: 0.8106 - model_6_acc: 0.8126 - model_7_acc: 0.8115 - model_8_acc: 0.8109 - val_loss: 2.0643 - val_model_5_loss: 0.5054 - val_model_6_loss: 0.5075 - val_model_7_loss: 0.5068 - val_model_8_loss: 0.5087 - val_model_5_acc: 0.8125 - val_model_6_acc: 0.8101 - val_model_7_acc: 0.8176 - val_model_8_acc: 0.8124
Epoch 42/100
114/115 [============================>.] - ETA: 0s - loss: 2.0695 - model_5_loss: 0.5129 - model_6_loss: 0.5069 - model_7_loss: 0.5067 - model_8_loss: 0.5070 - model_5_acc: 0.8118 - model_6_acc: 0.8130 - mo

Epoch 62/100
114/115 [============================>.] - ETA: 0s - loss: 2.0506 - model_5_loss: 0.5080 - model_6_loss: 0.5021 - model_7_loss: 0.5021 - model_8_loss: 0.5025 - model_5_acc: 0.8117 - model_6_acc: 0.8131 - model_7_acc: 0.8135 - model_8_acc: 0.8123
Epoch 00062: val_loss did not improve from 2.03028
115/115 [==============================] - 15s 134ms/step - loss: 2.0525 - model_5_loss: 0.5085 - model_6_loss: 0.5026 - model_7_loss: 0.5026 - model_8_loss: 0.5029 - model_5_acc: 0.8115 - model_6_acc: 0.8129 - model_7_acc: 0.8133 - model_8_acc: 0.8121 - val_loss: 2.0459 - val_model_5_loss: 0.5090 - val_model_6_loss: 0.4987 - val_model_7_loss: 0.5005 - val_model_8_loss: 0.5018 - val_model_5_acc: 0.8139 - val_model_6_acc: 0.8133 - val_model_7_acc: 0.8135 - val_model_8_acc: 0.8169
Epoch 63/100
114/115 [============================>.] - ETA: 0s - loss: 2.0478 - model_5_loss: 0.5081 - model_6_loss: 0.5015 - model_7_loss: 0.5012 - model_8_loss: 0.5011 - model_5_acc: 0.8123 - model_6_acc

114/115 [============================>.] - ETA: 0s - loss: 2.0209 - model_5_loss: 0.5016 - model_6_loss: 0.4954 - model_7_loss: 0.4940 - model_8_loss: 0.4941 - model_5_acc: 0.8120 - model_6_acc: 0.8143 - model_7_acc: 0.8145 - model_8_acc: 0.8140
Epoch 00082: val_loss did not improve from 2.01135
115/115 [==============================] - 15s 133ms/step - loss: 2.0186 - model_5_loss: 0.5010 - model_6_loss: 0.4949 - model_7_loss: 0.4934 - model_8_loss: 0.4935 - model_5_acc: 0.8122 - model_6_acc: 0.8146 - model_7_acc: 0.8147 - model_8_acc: 0.8143 - val_loss: 2.0114 - val_model_5_loss: 0.5008 - val_model_6_loss: 0.4936 - val_model_7_loss: 0.4905 - val_model_8_loss: 0.4907 - val_model_5_acc: 0.8162 - val_model_6_acc: 0.8121 - val_model_7_acc: 0.8128 - val_model_8_acc: 0.8136
Epoch 83/100
114/115 [============================>.] - ETA: 0s - loss: 2.0474 - model_5_loss: 0.5077 - model_6_loss: 0.5016 - model_7_loss: 0.5014 - model_8_loss: 0.5009 - model_5_acc: 0.8099 - model_6_acc: 0.8109 - mo

Epoch 93/100
114/115 [============================>.] - ETA: 0s - loss: 2.0203 - model_5_loss: 0.5018 - model_6_loss: 0.4947 - model_7_loss: 0.4943 - model_8_loss: 0.4938 - model_5_acc: 0.8139 - model_6_acc: 0.8158 - model_7_acc: 0.8155 - model_8_acc: 0.8154
Epoch 00093: val_loss did not improve from 2.01135
115/115 [==============================] - 15s 133ms/step - loss: 2.0190 - model_5_loss: 0.5015 - model_6_loss: 0.4943 - model_7_loss: 0.4940 - model_8_loss: 0.4935 - model_5_acc: 0.8143 - model_6_acc: 0.8162 - model_7_acc: 0.8158 - model_8_acc: 0.8157 - val_loss: 2.0156 - val_model_5_loss: 0.5023 - val_model_6_loss: 0.4961 - val_model_7_loss: 0.4920 - val_model_8_loss: 0.4895 - val_model_5_acc: 0.8242 - val_model_6_acc: 0.8195 - val_model_7_acc: 0.8164 - val_model_8_acc: 0.8190
Epoch 94/100
114/115 [============================>.] - ETA: 0s - loss: 2.0297 - model_5_loss: 0.5045 - model_6_loss: 0.4965 - model_7_loss: 0.4962 - model_8_loss: 0.4967 - model_5_acc: 0.8128 - model_6_acc